In [ ]:
!pip install datasets
!pip install trl
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

### DATASET

In [ ]:
from datasets import load_dataset, DatasetDict

ds = load_dataset("CShorten/ML-ArXiv-Papers")
ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/986 [00:00<?, ?B/s]

ML-Arxiv-Papers.csv:   0%|          | 0.00/147M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/117592 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
        num_rows: 117592
    })
})

In [ ]:
# indices = len(ds['train'])
# indices = int(indices * 0.8)
# train_ds = ds['train'].take(indices)
# test_ds = ds['train'].skip(indices)

indices_len = len(ds['train'])
indices = int(indices_len * 0.05)
train_ds = ds['train'].take(indices)
indices = int(indices_len * 0.95)
test_ds = ds['train'].skip(indices)

raw_datasets = DatasetDict({
    'train': train_ds,
    'test': test_ds
})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
        num_rows: 5879
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
        num_rows: 5880
    })
})

### DATA FORMATTING

In [ ]:
example = raw_datasets["train"][0]
print(example.keys())

dict_keys(['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'])


In [ ]:
messages = [
    {"role": "system", "content": "You are a reserach assistant and write abstracts for reserach topics"},
    {"role": "user", "content": example['title']},
    {"role": "assistant", "content": example['abstract']}
    ]
messages

[{'role': 'system',
  'content': 'You are a reserach assistant and write abstracts for reserach topics'},
 {'role': 'user', 'content': 'Learning from compressed observations'},
 {'role': 'assistant',
  'content': '  The problem of statistical learning is to construct a predictor of a random\nvariable $Y$ as a function of a related random variable $X$ on the basis of an\ni.i.d. training sample from the joint distribution of $(X,Y)$. Allowable\npredictors are drawn from some specified class, and the goal is to approach\nasymptotically the performance (expected loss) of the best predictor in the\nclass. We consider the setting in which one has perfect observation of the\n$X$-part of the sample, while the $Y$-part has to be communicated at some\nfinite bit rate. The encoding of the $Y$-values is allowed to depend on the\n$X$-values. Under suitable regularity conditions on the admissible predictors,\nthe underlying family of probability distributions and the loss function, we\ngive an infor

### TOKENIZER

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
HF_TOKEN = ""
model_id = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
tokenizer.chat_template

'{%- if tools %}\n    {{- \'<|im_start|>system\\n\' }}\n    {%- if messages[0][\'role\'] == \'system\' %}\n        {{- messages[0][\'content\'] }}\n    {%- else %}\n        {{- \'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\' }}\n    {%- endif %}\n    {{- "\\n\\n# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>" }}\n    {%- for tool in tools %}\n        {{- "\\n" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- "\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\"name\\": <function-name>, \\"arguments\\": <args-json-object>}\\n</tool_call><|im_end|>\\n" }}\n{%- else %}\n    {%- if messages[0][\'role\'] == \'system\' %}\n        {{- \'<|im_start|>system\\n\' + messages[0][\'content\'] + \'<|im_end|>\\n\' }}\n    {%- else %}\n       

In [ ]:
tokenizer.apply_chat_template(messages, tokenize=False)

'<|im_start|>system\nYou are a reserach assistant and write abstracts for reserach topics<|im_end|>\n<|im_start|>user\nLearning from compressed observations<|im_end|>\n<|im_start|>assistant\n  The problem of statistical learning is to construct a predictor of a random\nvariable $Y$ as a function of a related random variable $X$ on the basis of an\ni.i.d. training sample from the joint distribution of $(X,Y)$. Allowable\npredictors are drawn from some specified class, and the goal is to approach\nasymptotically the performance (expected loss) of the best predictor in the\nclass. We consider the setting in which one has perfect observation of the\n$X$-part of the sample, while the $Y$-part has to be communicated at some\nfinite bit rate. The encoding of the $Y$-values is allowed to depend on the\n$X$-values. Under suitable regularity conditions on the admissible predictors,\nthe underlying family of probability distributions and the loss function, we\ngive an information-theoretic charac

In [ ]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [ ]:
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

In [ ]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [ ]:
def apply_chat_template(example, tokenizer):
  messages = [
    {"role": "system", "content": "You are a reserach assistant and write abstracts for reserach topics"},
    {"role": "user", "content": example['title']},
    {"role": "assistant", "content": example['abstract']}
    ]
  example['text'] = tokenizer.apply_chat_template(messages, tokenize=False)
  return example

column_names = list(raw_datasets["train"].features)
raw_datasets = raw_datasets.map(apply_chat_template, fn_kwargs={"tokenizer": tokenizer}, remove_columns=column_names)

Map:   0%|          | 0/5879 [00:00<?, ? examples/s]

Map:   0%|          | 0/5880 [00:00<?, ? examples/s]

In [ ]:
raw_datasets['train'][0]

{'text': '<|im_start|>system\nYou are a reserach assistant and write abstracts for reserach topics<|im_end|>\n<|im_start|>user\nLearning from compressed observations<|im_end|>\n<|im_start|>assistant\n  The problem of statistical learning is to construct a predictor of a random\nvariable $Y$ as a function of a related random variable $X$ on the basis of an\ni.i.d. training sample from the joint distribution of $(X,Y)$. Allowable\npredictors are drawn from some specified class, and the goal is to approach\nasymptotically the performance (expected loss) of the best predictor in the\nclass. We consider the setting in which one has perfect observation of the\n$X$-part of the sample, while the $Y$-part has to be communicated at some\nfinite bit rate. The encoding of the $Y$-values is allowed to depend on the\n$X$-values. Under suitable regularity conditions on the admissible predictors,\nthe underlying family of probability distributions and the loss function, we\ngive an information-theoret

### FINETUNING

In [ ]:
from transformers import BitsAndBytesConfig
import torch

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float32,
            llm_int8_enable_fp32_cpu_offload=True,
)
# device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None
device_map = "auto"
# device_name = 'cuda' if cuda.is_available() else 'cpu'

model_kwargs = dict(
    attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=quantization_config,
)

model = AutoModelForCausalLM.from_pretrained(
   model_id, device_map=device_map, quantization_config=quantization_config, token=HF_TOKEN
)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# based on config
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, peft_config)

train_p, tot_p = model.get_nb_trainable_parameters()
print(f'Trainable parameters:      {train_p/1e6:.2f}M')
print(f'Total parameters:          {tot_p/1e6:.2f}M')
print(f'% of trainable parameters: {100*train_p/tot_p:.2f}%')

Trainable parameters:      29.49M
Total parameters:          3115.43M
% of trainable parameters: 0.95%


In [ ]:
tokenizer.model_max_length

131072

In [ ]:
from trl import SFTTrainer, SFTConfig
# from transformers import TrainingArguments

# path where the Trainer will save its checkpoints and logs
output_dir = 'data/research-7b-instruct-sft-lora'

sft_config = SFTConfig(
    ## GROUP 1: Memory usage
    # These arguments will squeeze the most out of your GPU's RAM
    # Checkpointing
    gradient_checkpointing=True,    # this saves a LOT of memory
    # Set this to avoid exceptions in newer versions of PyTorch
    gradient_checkpointing_kwargs={'use_reentrant': False},
    # Gradient Accumulation / Batch size
    # Actual batch (for updating) is same (1x) as micro-batch size
    # gradient_accumulation_steps=1,
    # The initial (micro) batch size to start off with
    per_device_train_batch_size=8,
    per_device_eval_batch_size=2,
    # If batch size would cause OOM, halves its size until it works
    # auto_find_batch_size=True,

    ## GROUP 2: Dataset-related
    # max_seq_length=tokenizer.model_max_length,
    max_seq_length=512,
    # Dataset
    # packing a dataset means no padding is needed
    packing=True,

    ## GROUP 3: These are typical training parameters
    num_train_epochs=1,
    learning_rate=2e-2,
    # Optimizer
    # 8-bit Adam optimizer - doesn't help much if you're using LoRA!
    optim='paged_adamw_8bit',

    ## GROUP 4: Logging parameters
    logging_steps=100,
    logging_dir='./logs',
    output_dir=output_dir,
    report_to='none'
)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    processing_class=tokenizer,
    train_dataset=raw_datasets['train'],
    eval_dataset=raw_datasets['test'],
)

Converting train dataset to ChatML:   0%|          | 0/5879 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/5879 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5879 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/5879 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/5880 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/5880 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/5880 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/5880 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
torch.cuda.empty_cache()

In [ ]:
train_result = trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
100,13.641700
200,9.855700
300,10.768600


Step,Training Loss
100,13.641700
200,9.855700
300,10.768600


### METRICS

In [ ]:
metrics = train_result.metrics
metrics

{'train_runtime': 6798.2048,
 'train_samples_per_second': 0.42,
 'train_steps_per_second': 0.053,
 'total_flos': 2.457774468366336e+16,
 'train_loss': 10.909106684666053}

In [ ]:
# save model
trainer.save_model("fine_tuned/model")

In [ ]:
def gen_prompt(tokenizer, sentence):
    converted_sample = [
        {"role": "system", "content": "You are a reserach assistant and write abstracts for reserach topics"},
        {"role": "user", "content": sentence}]
    prompt = tokenizer.apply_chat_template(
        converted_sample, tokenize=False, add_generation_prompt=True
    )
    return prompt

sentence = 'Parts of human brain is psychology'
prompt = gen_prompt(tokenizer, sentence)
print(prompt)

<|im_start|>system
You are a reserach assistant and write abstracts for reserach topics<|im_end|>
<|im_start|>user
Parts of human brain is psychology<|im_end|>
<|im_start|>assistant



In [ ]:
def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=False):
    tokenized_input = tokenizer(
        prompt, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)

    model.eval()
    gen_output = model.generate(**tokenized_input,
                                eos_token_id=tokenizer.eos_token_id,
                                max_new_tokens=max_new_tokens)

    output = tokenizer.batch_decode(gen_output, skip_special_tokens=skip_special_tokens)
    return output[0]

In [ ]:
sentence = 'Parts of human brain in psychology'
prompt = gen_prompt(tokenizer, sentence)
# print(prompt)
print(generate(model, tokenizer, prompt))

<|im_start|>system
You are a reserach assistant and write abstracts for reserach topics<|im_end|>
<|im_start|>user
Parts of human brain is psychology<|im_end|>
<|im_start|>assistant
 the, a of to. the.

 (
 a

 the and a,,. a a a

 in
 and.




 the

 res.





 res
 and
 the
achachach the We, the to We





In [ ]:
model = trainer.model

In [ ]:
def generate_text(prompt, max_new_tokens=300, temperature=0.1, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        eos_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)
prompt = "The problem of statistical learning is to construct a predictor of a random\nvariable"
generated_text = generate_text(prompt)

print(generated_text)

The problem of statistical learning is to construct a predictor of a random
variable Whites PREF"]]
(ENDnty#End"]
.asm Phaservana PREF"]]
vanavana Desmond"]]
 PREFosexvana周转辐淞矶_Internal有名 PREF"]]
亲爱的vana"]}
 PREF"]]
.getEndosex['__�� PREF淞"]

"]
vanaudasvana说的是"]
 Desmondозвosex辗dd optimizationdd abstract rating storageockerdd models留学生dd algorithms quadratic sensneg ratingorsธรรมdns posthink the algorithms alternatively development
achocker親 body meeting body downs teamærors storage Suicidedns stalled bodyors plans developmentprodnsdd Ditdnsicular development付け.


ors body_ANTocker親ors rating_lock Chess development data共享pro assistant in previous for approximation cop親 Electroocker aboveors rating meeting plans downs body lower body the res algorithms itemaz We algorithms development body meetingitem paperors Abr�dns親 posit. body that alternatively withaz body親 pos body previous body the

 practical todnsocker developmentocker plans development downs is We optimal to great petitionpro

In [1]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("fine_tuned/model")
# model = AutoModelForCausalLM.from_pretrained("fine_tuned/model", load_in_4bit=True, device_map="auto")

In [ ]:
# import torch

# # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
# messages = [
#     {
#         "role": "system",
#         "content": "You are a friendly chatbot who always responds in the style of a pirate",
#     },
#     {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
# ]

# # prepare the messages for the model
# input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# # inference
# outputs = model.generate(
#         input_ids=input_ids,
#         max_new_tokens=256,
#         do_sample=True,
#         temperature=0.7,
#         top_k=50,
#         top_p=0.95
# )
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])